In [1]:
import warnings

# Filter out specific warnings
warnings.filterwarnings("ignore")

# Your code that generates warnings

# Restore the default warning behavior
warnings.filterwarnings("default")

In [2]:
import dask.dataframe as dd

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import time
import pickle
import os
import matplotlib.pyplot as plt

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_curve, roc_auc_score

In [5]:
import cv2
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
df = dd.read_csv('/kaggle/input/water-quality-prediction/dataset.csv')
df.head()

,Index,pH,Iron,Nitrate,Chloride,Lead,Zinc,Color,Turbidity,Fluoride,...,Chlorine,Manganese,Total Dissolved Solids,Source,Water Temperature,Air Temperature,Month,Day,Time of Day,Target
0,0,8.332988,0.000083,8.605777,122.799772,3.713298e-52,3.434827,Colorless,0.022683,0.607283,...,3.708178,2.269945e-15,332.118789,NaN,NaN,43.493324,January,29.0,4.0,0
1,1,6.917863,0.000081,3.734167,227.029851,7.849262e-94,1.245317,Faint Yellow,0.019007,0.622874,...,3.292038,8.024076e-07,284.641984,Lake,15.348981,71.220586,November,26.0,16.0,0
2,2,5.443762,0.020106,3.816994,230.995630,5.286616e-76,0.528280,Light Yellow,0.319956,0.423423,...,3.560224,7.007989e-02,570.054094,River,11.643467,44.891330,January,31.0,8.0,0
3,3,7.955339,0.143988,8.224944,178.129940,3.997118e-176,4.027879,Near Colorless,0.166319,0.208454,...,3.516907,2.468295e-02,100.043838,Ground,10.092392,60.843233,April,1.0,21.0,0
4,4,8.091909,0.002167,9.925788,186.540872,4.171069e-132,3.807511,Light Yellow,0.004867,0.222912,...,3.177849,3.296139e-03,168.075545,Spring,15.249416,69.336671,June,29.0,7.0,0


In [7]:
df = df.drop(['Index'], axis = 1)

In [8]:
rem=['Month', 'Day', 'Time of Day']

In [9]:
df1 = df.drop(rem, axis=1)

In [10]:
df1['Color'] = df1.Color.replace(['Colorless', 'Near Colorless','Faint Yellow', 'Light Yellow','Yellow'],[0,1,2,3,4])

In [11]:
df1['Source']=df1['Source'].replace(['Stream', 'Ground','Well','Aquifer','Reservoir'
,'River', 'Spring','Lake'], [0,1,2,3,4,5,6,7])

In [12]:
X = df1.drop(['Target'], axis = 1)
y = df['Target']

In [13]:
X_pd = X.compute().reset_index(drop=True)
y_pd = y.compute().reset_index(drop=True)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_pd, y_pd, test_size=0.2)

imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()
# Impute missing values in X_train and X_test
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
# Scale the features in X_train and X_test
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# LR

In [17]:
start_time = time.time()
model_lr = LogisticRegression()# Logistic Regression
# Fit the logistic regression model on the training data
model_lr.fit(X_train_scaled, y_train)
train_time = time.time()-start_time
# Make predictions on the test data
start_time = time.time()
y_pred = model_lr.predict(X_test_scaled)
predict_time = time.time()-start_time
print("Train Time : ", train_time, "\nTest Time : ", predict_time)

Train Time :  6.3545331954956055 
Test Time :  0.07294845581054688


In [18]:
accuracy = model_lr.score(X_test_scaled, y_test)
# Save the trained model to the Kaggle Outputs folder
filename = 'Logistic_Regression.pkl'
output_path = '/kaggle/working/'  # This is the path to the Kaggle Outputs folder

with open(os.path.join(output_path, filename), 'wb') as file:
    pickle.dump(model_lr, file)
accuracy

0.7906836588831839

In [19]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)

Precision: 0.7202966772151899
Recall: 0.5047497609180746


# RF

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
print('Random_Forest')
# Create a Random Forest classifier
start_time = time.time()
model_rf = RandomForestClassifier(n_estimators=50, random_state=42)

# Fit the model on the training data
model_rf.fit(X_train_scaled, y_train)
train_time = time.time()-start_time

Random_Forest


In [21]:
# Make predictions on the test data
start_time = time.time()
y_pred = model_rf.predict(X_test_scaled)
predict_time = time.time()-start_time

print("Train Time : ", train_time, "\nTest Time : ", predict_time)

Train Time :  5859.507010221481 
Test Time :  46.285792112350464


In [22]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)

Precision: 0.7394037012660677
Recall: 0.9736078500863864
